In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  Gold.1  \
Afghanistan        13     0       0       2      2         0       0   
Algeria            12     5       2       8     15         3       0   
Argentina          23    18      24      28     70        18       0   
Armenia             5     1       2       9     12         6       0   
Australasia         2     3       4       5     12         0       0   

             Silver.1  Bronze.1  Total.1  # Games  Gold.2  Silver.2  Bronze.2  \
Afghanistan         0         0        0       13       0         0         2   
Algeria             0         0        0       15       5         2         8   
Argentina           0         0        0       41      18        24        28   
Armenia             0         0        0       11       1         2         9   
Australasia         0         0        0        2       3         4         5   

             Combined total   ID  
Afghanistan               2  AFG  
Algeria                  15  ALG  
Argentina                70  ARG  
Armenia                  12  ARM  
Australasia              12  ANZ

In [4]:
# first country in df
df.sort_index()
df.index[0]
df.iloc[0]df['Gold'].idxmax()

str

In [22]:
# which country has most gold in summer games
def ans():
    df1 = df.copy()
    df1['Country'] = df.index
    df1.reset_index()
    df1 = df1.set_index(['Gold'])
    df1 = df1.sort_index()
    length = len(df1.index)
    return df1.iloc[length-1]['Country']

ans()

'United States'

In [45]:
# which country has biggest difference between their summer and winter gold medal counts?
def sub(a,b):
    return abs(a-b)

def ans():
    df2 = df.copy()
    difference = list(map(sub,df2['Gold'],df2['Gold.1']))
    df2['Diff'] = difference
    df2 = df2[['Gold','Gold.1','Diff']]
    df2['Country'] = df2.index
    df2.reset_index()
    df2 = df2.set_index(['Diff'])
    df2 = df2.sort_index(ascending=False)
    return df2.iloc[0]['Country']

ans()
    

'United States'

## 3 

Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count?

(Summer Gold−Winter Gold)/Total Gold

Only include countries that have won at least 1 gold in both summer and winter.


In [71]:
def add(a,b):
    return a+b

def rel(a,b,c):
    return (a-b)/c

def ans():
    df3 = df.loc[:,['Gold','Gold.1','Gold.2']]
    df3 = df3.where((df['Gold']>=1) & (df3['Gold.1']>=1))
    df3 = df3.dropna()
    df3['Rscore'] = list(map(rel,df3['Gold'],df3['Gold.1'],df3['Gold.2']))
    df3['Country'] = df3.index
    df3.reset_index()
    df3 = df3.set_index(['Rscore'])
    df3 = df3.sort_index(ascending=False)
    return df3.iloc[0]['Country']

ans()

'Bulgaria'

# 4

Write a function that creates a Series called "Points" which is a weighted value where each gold medal (Gold.2) counts for 3 points, silver medals (Silver.2) for 2 points, and bronze medals (Bronze.2) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

This function should return a Series named Points of length 146

In [73]:
def add(a, b, c):
    return 3*a+2*b+c

def ans():
    df4 = df.loc[:,['Gold.2','Silver.2','Bronze.2']]
    total = list(map(add,df4['Gold.2'],df4['Silver.2'],df4['Bronze.2']))
    series = pd.Series(total,index=df4.index)
    return series

ans()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Length: 146, dtype: int64


## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.



In [2]:
census_df = pd.read_csv('census.csv')
#census_df.head()
census_df.where((census_df['SUMLEV']==50)).dropna().head()

SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME         CTYNAME  \
1    50.0     3.0       6.0    1.0     1.0  Alabama  Autauga County   
2    50.0     3.0       6.0    1.0     3.0  Alabama  Baldwin County   
3    50.0     3.0       6.0    1.0     5.0  Alabama  Barbour County   
4    50.0     3.0       6.0    1.0     7.0  Alabama     Bibb County   
5    50.0     3.0       6.0    1.0     9.0  Alabama   Blount County   

   CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  ...  RDOMESTICMIG2011  \
1        54571.0            54571.0          54660.0  ...          7.242091   
2       182265.0           182265.0         183193.0  ...         14.832960   
3        27457.0            27457.0          27341.0  ...         -4.728132   
4        22915.0            22919.0          22861.0  ...         -5.527043   
5        57322.0            57322.0          57373.0  ...          1.807375   

   RDOMESTICMIG2012  RDOMESTICMIG2013  RDOMESTICMIG2014  RDOMESTICMIG2015  \
1         -2.915927         -3.012349          2.265971         -2.530799   
2         17.647293         21.845705         19.243287         17.197872   
3         -2.500690         -7.056824         -3.904217        -10.543299   
4         -5.068871         -6.201001         -0.177537          0.177258   
5         -1.177622         -1.748766         -2.062535         -1.369970   

   RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
1     7.606016    -2.626146    -2.722002     2.592270    -2.187333  
2    15.844176    18.559627    22.727626    20.317142    18.293499  
3    -4.874741    -2.758113    -7.167664    -3.978583   -10.543299  
4    -5.088389    -4.363636    -5.403729     0.754533     1.107861  
5     1.859511    -0.848580    -1.402476    -1.577232    -0.884411  

[5 rows x 100 columns]

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [95]:
def ans():
    df5 = census_df.loc[:,['STNAME','COUNTY']]
    df5 = df5.groupby('STNAME').count()
    df5 = df5.sort_values(by=['COUNTY'],ascending=False)
    return df5.index[0]

ans()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [26]:
def calcpop(count,county):
    if county != 'District of Columbia':
        count = count.sort_values(by=['CENSUS2010POP'],ascending=False)
        pop = np.array(count['CENSUS2010POP'])
        pop = np.sum(pop[:3])
        return pop
    else:
        return count.loc['CENSUS2010POP']

def ans():
    df6 = census_df.copy() 
    df6 = df6.loc[:,['SUMLEV','STNAME','COUNTY','CENSUS2010POP']]
    df6 = df6.where((df6['SUMLEV']==50)).dropna().drop('SUMLEV',axis=1)
    df6.reset_index()
    df6 = df6.set_index(['STNAME'])
    population = []
    sample = df6.loc['District of Columbia']
    for county in set(df6.index):
        population.append(calcpop(df6.loc[county],county))
    series = pd.Series(list(set(df6.index)),index=population)
    series = series.sort_index(ascending=False)
    series = list(series)
    return series[:3]
ans()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [6]:
def calc(values):
    values = np.array(values)
    return np.abs(np.max(values)-np.min(values))

def ans():
    df7 = census_df.copy()
    df7 = df7.loc[:,['SUMLEV','STNAME','CTYNAME','POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']]
    df7 = df7.where((df7['SUMLEV']==50)).dropna().drop('SUMLEV',axis=1)
    df7.reset_index()
    df7 = df7.set_index(['STNAME','CTYNAME'])
    deviation = []
    for county in df7.index:
        deviation.append(calc(df7.loc[county]))
    df7['Deviation'] = deviation
    df7 = df7.sort_values(by=['Deviation'],ascending=False)
    df7 = df7.drop(['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015'],axis=1)
    return df7.index[0][1]
    

ans()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [196]:
def ans():
    df8 = census_df.where(((census_df['REGION']==1) | (census_df['REGION'] == 2))).dropna()
    df8 = df8.where((df8['POPESTIMATE2015']>df8['POPESTIMATE2014'])).dropna()
    df8 = df8.where((df8['CTYNAME'].str.contains("Washington"))).dropna()
    df8 = df8.loc[:,['STNAME','CTYNAME']]
    df8 = df8.sort_index()
    return df8

ans()

STNAME            CTYNAME
896           Iowa  Washington County
1419     Minnesota  Washington County
2345  Pennsylvania  Washington County
2355  Rhode Island  Washington County
3163     Wisconsin  Washington County